In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

In [18]:
data = pd.read_csv(r'E:\Info_Srping\FWI_Prediction\Data\processed\\test_train_split_after_outlier\train.csv')
data

,temperature,dew_point_temperature,relative_humidity,wind_speed,wind_direction,atmospheric_pressure,visibility,precipitation,rain_days,snow_on_ground,...,duff_moisture_code,drought_code,build_up_index,initial_spread_index,fire_weather_index,daily_severity_rating,country,latitude,longitude,elevation
0,21.7,3.3,28.0,25.9,360,1020.40,16.1,0.00,8,0.0,...,52.9,426.5,80.8,26.9,53.8,31.5,United States,37.262524,-98.546196,472.1
1,0.1,-0.1,98.0,6.2,104,1015.97,25.3,0.00,4,0.0,...,37.6,646.4,65.7,1.5,5.6,0.6,Canada,49.550000,-111.050000,845.0
2,1.7,-8.3,47.8,24.1,330,1029.50,16.1,1.54,0,0.0,...,4.5,318.3,8.8,2.3,2.0,0.1,United States,39.771942,-94.909706,251.9
3,31.0,2.0,15.6,14.8,320,1019.60,16.1,0.00,18,0.0,...,146.5,688.1,191.2,21.3,62.2,40.7,United States,43.564360,-116.222860,875.3
4,24.0,7.4,36.0,5.6,271,1021.50,14.5,1.30,0,0.0,...,48.5,289.5,68.3,3.4,11.7,2.1,Canada,53.030000,-122.510000,545.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759584,25.4,16.0,55.0,10.6,231,1010.67,24.7,2.51,0,0.0,...,10.1,207.8,18.0,1.6,2.2,0.1,Canada,45.840000,-75.650000,194.6
759585,19.4,9.4,52.6,3.7,110,1012.30,42.2,0.70,0,0.0,...,12.5,45.2,14.8,2.2,2.8,0.2,Canada,47.320000,-71.150000,672.8
759586,16.4,7.5,55.0,14.0,223,1003.80,25.0,0.00,4,0.0,...,32.9,116.6,38.5,6.4,14.0,2.9,Canada,64.050000,-139.133330,370.0
759587,21.4,7.9,43.1,14.2,318,1010.63,21.4,0.40,0,0.0,...,85.1,444.3,115.1,9.8,33.1,13.3,Canada,53.340000,-110.880000,623.0


In [19]:
train_data = data.iloc[:20000]       
train_data

,temperature,dew_point_temperature,relative_humidity,wind_speed,wind_direction,atmospheric_pressure,visibility,precipitation,rain_days,snow_on_ground,...,duff_moisture_code,drought_code,build_up_index,initial_spread_index,fire_weather_index,daily_severity_rating,country,latitude,longitude,elevation
0,21.7,3.3,28.0,25.9,360,1020.40,16.1,0.00,8,0.0,...,52.9,426.5,80.8,26.9,53.8,31.5,United States,37.262524,-98.546196,472.1
1,0.1,-0.1,98.0,6.2,104,1015.97,25.3,0.00,4,0.0,...,37.6,646.4,65.7,1.5,5.6,0.6,Canada,49.550000,-111.050000,845.0
2,1.7,-8.3,47.8,24.1,330,1029.50,16.1,1.54,0,0.0,...,4.5,318.3,8.8,2.3,2.0,0.1,United States,39.771942,-94.909706,251.9
3,31.0,2.0,15.6,14.8,320,1019.60,16.1,0.00,18,0.0,...,146.5,688.1,191.2,21.3,62.2,40.7,United States,43.564360,-116.222860,875.3
4,24.0,7.4,36.0,5.6,271,1021.50,14.5,1.30,0,0.0,...,48.5,289.5,68.3,3.4,11.7,2.1,Canada,53.030000,-122.510000,545.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,23.6,10.0,40.0,9.0,135,1020.25,3.8,1.30,0,0.0,...,34.6,296.4,53.5,4.0,11.6,2.1,United Kingdom,54.367000,-1.617000,53.0
19996,23.0,20.8,87.0,16.1,184,1013.10,18.1,4.50,0,0.0,...,22.3,212.6,35.3,0.6,0.8,0.0,Canada,46.460000,-61.990000,7.7
19997,13.0,-2.0,35.0,14.8,280,1017.70,16.1,0.00,4,0.0,...,13.4,118.6,20.9,6.1,9.7,1.5,United States,36.894608,-76.201230,8.0
19998,12.8,3.6,53.0,10.2,218,1023.70,17.3,9.80,0,0.0,...,4.5,141.5,8.4,0.2,0.1,0.0,Canada,46.710000,-79.100000,181.4


In [20]:
train_data = train_data.drop(['wind_direction','snow_on_ground','daily_severity_rating','latitude','longitude', 'elevation','visibility'], axis = 1)
train_data

,temperature,dew_point_temperature,relative_humidity,wind_speed,atmospheric_pressure,precipitation,rain_days,fine_fuel_moisture_code,duff_moisture_code,drought_code,build_up_index,initial_spread_index,fire_weather_index,country
0,21.7,3.3,28.0,25.9,1020.40,0.00,8,93.7,52.9,426.5,80.8,26.9,53.8,United States
1,0.1,-0.1,98.0,6.2,1015.97,0.00,4,79.9,37.6,646.4,65.7,1.5,5.6,Canada
2,1.7,-8.3,47.8,24.1,1029.50,1.54,0,72.8,4.5,318.3,8.8,2.3,2.0,United States
3,31.0,2.0,15.6,14.8,1019.60,0.00,18,96.1,146.5,688.1,191.2,21.3,62.2,United States
4,24.0,7.4,36.0,5.6,1021.50,1.30,0,86.4,48.5,289.5,68.3,3.4,11.7,Canada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,23.6,10.0,40.0,9.0,1020.25,1.30,0,86.4,34.6,296.4,53.5,4.0,11.6,United Kingdom
19996,23.0,20.8,87.0,16.1,1013.10,4.50,0,54.9,22.3,212.6,35.3,0.6,0.8,Canada
19997,13.0,-2.0,35.0,14.8,1017.70,0.00,4,87.3,13.4,118.6,20.9,6.1,9.7,United States
19998,12.8,3.6,53.0,10.2,1023.70,9.80,0,45.7,4.5,141.5,8.4,0.2,0.1,Canada


In [21]:
# Load saved PowerTransformer dictionary
with open(r'E:\Info_Srping\FWI_Prediction\models\Pkl_files_after_outlier_remover\power_transformers.pkl', 'rb') as f:
    power_transformers = pickle.load(f)

# Load saved StandardScaler
with open(r'E:\Info_Srping\FWI_Prediction\models\Pkl_files_after_outlier_remover\standard_scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

# Load saved OneHotEncoding
with open(r'E:\Info_Srping\FWI_Prediction\models\Pkl_files_after_outlier_remover\one_hot_encoder.pkl', 'rb') as f:
    ohe = pickle.load(f)


In [22]:
col = ['fine_fuel_moisture_code','duff_moisture_code', 'drought_code','build_up_index', 'initial_spread_index','fire_weather_index','wind_speed']
for col, pt in power_transformers.items():
    # Transform test column using fitted transformer
    transformed = pt.transform(train_data[col].values.reshape(-1, 1))
    train_data[col + '_transformed'] = transformed.flatten()


In [23]:
numeric_cols = train_data.select_dtypes(include=['float64', 'int']).columns
train_data[numeric_cols] = scaler.transform(train_data[numeric_cols])

In [24]:
train_data = train_data[['temperature', 'dew_point_temperature', 'relative_humidity',
        'atmospheric_pressure', 'precipitation', 'rain_days',
       'country', 'fine_fuel_moisture_code_transformed',
       'duff_moisture_code_transformed', 'drought_code_transformed',
       'build_up_index_transformed', 'initial_spread_index_transformed',
       'fire_weather_index_transformed', 'wind_speed_transformed']]

In [25]:
train_data['FFMC__TEMP'] = train_data['temperature'] * train_data['fine_fuel_moisture_code_transformed']
train_data['FFMC_RH'] = train_data['relative_humidity'] * train_data['fine_fuel_moisture_code_transformed']
train_data['RH_TEMP'] = train_data['temperature'] * train_data['relative_humidity']
train_data['RD_FFMC'] =  train_data['fine_fuel_moisture_code_transformed'] * train_data['rain_days']
train_data['DC_RD'] =  train_data['drought_code_transformed'] * train_data['rain_days']
train_data['WS_TENP'] =  train_data['wind_speed_transformed'] * train_data['fine_fuel_moisture_code_transformed']
train_data['IST_WS'] =  train_data['wind_speed_transformed'] * train_data['initial_spread_index_transformed']
train_data['ISI_FFMC'] =  train_data['initial_spread_index_transformed'] * train_data['fine_fuel_moisture_code_transformed']
train_data['ISI_RH'] =  train_data['initial_spread_index_transformed'] * train_data['relative_humidity']
train_data['DMC_TEMP'] =  train_data['duff_moisture_code_transformed'] * train_data['temperature']
train_data['DMC_RD'] =  train_data['duff_moisture_code_transformed'] * train_data['rain_days']
train_data['RD_TEMP'] =  train_data['rain_days'] * train_data['temperature']
train_data['DMC_DC'] =  train_data['drought_code_transformed'] * train_data['duff_moisture_code_transformed']
train_data['BUI_DC'] =  train_data['drought_code_transformed'] * train_data['build_up_index_transformed']
train_data['BUI_DMC'] =  train_data['duff_moisture_code_transformed'] * train_data['build_up_index_transformed']

C:\Users\DELL\AppData\Local\Temp\ipykernel_9392\1968526459.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['FFMC__TEMP'] = train_data['temperature'] * train_data['fine_fuel_moisture_code_transformed']
C:\Users\DELL\AppData\Local\Temp\ipykernel_9392\1968526459.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['FFMC_RH'] = train_data['relative_humidity'] * train_data['fine_fuel_moisture_code_transformed']
C:\Users\DELL\AppData\Local\Temp\ipykernel_9392\1968526459.py:3: SettingWithC

In [26]:
encoded_array = ohe.transform(train_data[['country']])  
encoded_df = pd.DataFrame(encoded_array, columns=ohe.get_feature_names_out(['country']))
new_data_encoded = pd.concat([train_data.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)


In [27]:
new_data_encoded = new_data_encoded.drop(['country'], axis =1)
new_data_encoded

,temperature,dew_point_temperature,relative_humidity,atmospheric_pressure,precipitation,rain_days,fine_fuel_moisture_code_transformed,duff_moisture_code_transformed,drought_code_transformed,build_up_index_transformed,...,DMC_RD,RD_TEMP,DMC_DC,BUI_DC,BUI_DMC,country_Canada,country_France,country_Spain,country_United Kingdom,country_United States
0,0.642664,-0.636352,-1.483321,0.846691,-0.368303,1.445606,1.360353,1.191756,1.011975,1.246469,...,1.722810,0.929039,1.206027,1.261395,1.485487,0,0,0,0,1
1,-2.306720,-1.127052,1.948817,0.183518,-0.368303,0.424063,0.109223,0.836119,1.655168,1.011166,...,0.354567,-0.978194,1.383918,1.673651,0.845455,1,0,0,0,0
2,-2.088247,-2.310505,-0.512516,2.208964,-0.129282,-0.597480,-0.393264,-0.964458,0.618773,-0.807440,...,0.576245,1.247687,-0.596781,-0.499622,0.778742,0,0,0,0,1
3,1.912537,-0.823972,-2.091300,0.726931,-0.368303,3.999464,1.618197,2.359785,1.761348,2.336418,...,9.437875,7.649125,4.156402,4.115246,5.513444,0,0,0,0,1
4,0.956719,-0.044625,-1.091077,1.011361,-0.166532,-0.597480,0.651045,1.099615,0.500810,1.054569,...,-0.656998,-0.571621,0.550698,0.528138,1.159619,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.902100,0.330616,-0.894954,0.824236,-0.166532,-0.597480,0.651045,0.752180,0.529695,0.787036,...,-0.449413,-0.538987,0.398426,0.416889,0.591992,0,0,0,1,0
19996,0.820173,1.889310,1.409481,-0.246122,0.330136,-0.597480,-1.297142,0.326188,0.146128,0.361305,...,-0.194891,-0.490037,0.047665,0.052797,0.117854,1,0,0,0,0
19997,-0.545282,-1.401266,-1.140107,0.442500,-0.368303,0.424063,0.732577,-0.130493,-0.416285,-0.124360,...,-0.055337,-0.231234,0.054322,0.051769,0.016228,0,0,0,0,1
19998,-0.572592,-0.593055,-0.257557,1.340702,1.152741,-0.597480,-1.590669,-0.964458,-0.259739,-0.840084,...,0.576245,0.342112,0.250508,0.218203,0.810226,1,0,0,0,0


In [28]:
new_data_encoded['ISI_Squared'] = new_data_encoded['initial_spread_index_transformed'] ** 2
new_data_encoded['BUI_Squared']   = new_data_encoded['build_up_index_transformed'] ** 2
new_data_encoded['DMC_Squared']   = new_data_encoded['duff_moisture_code_transformed'] ** 2

In [29]:
# Prepare features and target from your new test data
X = new_data_encoded.drop('fire_weather_index_transformed', axis = 1)
y = new_data_encoded['fire_weather_index_transformed']

In [30]:
def calculate_vif(X: pd.DataFrame):
    """
    Calculate VIF for each feature in the DataFrame.
    """
    # Add intercept for statsmodels
    X_with_const = add_constant(X)

    vif_data = pd.DataFrame()
    vif_data["Feature"] = X_with_const.columns
    vif_data["VIF"] = [
        variance_inflation_factor(X_with_const.values, i)
        for i in range(X_with_const.shape[1])
    ]

    # Drop the intercept row
    return vif_data.drop(index=0).reset_index(drop=True)

In [31]:
# Example usage
vif_df = calculate_vif(X)
print(vif_df)

C:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
C:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                                Feature          VIF
0                           temperature    81.525544
1                 dew_point_temperature    79.941145
2                     relative_humidity    55.784604
3                  atmospheric_pressure     1.161030
4                         precipitation     1.951915
5                             rain_days     7.330910
6   fine_fuel_moisture_code_transformed   132.931422
7        duff_moisture_code_transformed  1328.968333
8              drought_code_transformed    64.993193
9            build_up_index_transformed  1802.735661
10     initial_spread_index_transformed   132.980689
11               wind_speed_transformed    16.336297
12                           FFMC__TEMP     4.567242
13                              FFMC_RH    19.008972
14                              RH_TEMP     3.473447
15                              RD_FFMC     9.463397
16                                DC_RD     3.302707
17                              WS_TENP    17.

In [34]:
# Prepare features and target from your new test data
X = new_data_encoded.drop(['fire_weather_index_transformed','BUI_DMC'], axis = 1)
y = new_data_encoded['fire_weather_index_transformed']

In [35]:
# Example usage
vif_df = calculate_vif(X)
print(vif_df)

C:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
C:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                                Feature          VIF
0                           temperature    81.525526
1                 dew_point_temperature    79.939739
2                     relative_humidity    55.784390
3                  atmospheric_pressure     1.161030
4                         precipitation     1.951791
5                             rain_days     7.330440
6   fine_fuel_moisture_code_transformed   132.903379
7        duff_moisture_code_transformed  1294.472604
8              drought_code_transformed    61.562446
9            build_up_index_transformed  1749.374844
10     initial_spread_index_transformed   132.947182
11               wind_speed_transformed    16.333192
12                           FFMC__TEMP     4.567186
13                              FFMC_RH    19.007777
14                              RH_TEMP     3.473022
15                              RD_FFMC     9.463044
16                                DC_RD     3.293553
17                              WS_TENP    17.

In [37]:
# Prepare features and target from your new test data
X = new_data_encoded.drop(['fire_weather_index_transformed','BUI_DMC','build_up_index_transformed','country_United States'], axis = 1)
y = new_data_encoded['fire_weather_index_transformed']
# Example usage
vif_df = calculate_vif(X)
print(vif_df)

                                Feature         VIF
0                           temperature   81.452326
1                 dew_point_temperature   79.876227
2                     relative_humidity   55.745371
3                  atmospheric_pressure    1.160931
4                         precipitation    1.951468
5                             rain_days    7.328912
6   fine_fuel_moisture_code_transformed  132.842674
7        duff_moisture_code_transformed    4.379672
8              drought_code_transformed    2.628063
9      initial_spread_index_transformed  132.885836
10               wind_speed_transformed   16.328979
11                           FFMC__TEMP    4.567186
12                              FFMC_RH   19.004456
13                              RH_TEMP    3.472407
14                              RD_FFMC    9.456330
15                                DC_RD    3.293217
16                              WS_TENP   16.998457
17                               IST_WS   46.280631
18          

In [38]:
# Prepare features and target from your new test data
X = new_data_encoded.drop(['fire_weather_index_transformed','BUI_DMC','build_up_index_transformed','country_United States','initial_spread_index_transformed'], axis = 1)
y = new_data_encoded['fire_weather_index_transformed']
# Example usage
vif_df = calculate_vif(X)
print(vif_df)


                                Feature         VIF
0                           temperature   80.920932
1                 dew_point_temperature   79.616297
2                     relative_humidity   55.694962
3                  atmospheric_pressure    1.160796
4                         precipitation    1.912264
5                             rain_days    7.308016
6   fine_fuel_moisture_code_transformed   10.491879
7        duff_moisture_code_transformed    4.347457
8              drought_code_transformed    2.627928
9                wind_speed_transformed    1.672335
10                           FFMC__TEMP    4.560660
11                              FFMC_RH   18.984185
12                              RH_TEMP    3.453042
13                              RD_FFMC    9.283748
14                                DC_RD    3.292402
15                              WS_TENP   11.092807
16                               IST_WS   19.880638
17                             ISI_FFMC   64.128117
18          

In [39]:
# Prepare features and target from your new test data
X = new_data_encoded.drop(['fire_weather_index_transformed','BUI_DMC','build_up_index_transformed','country_United States',
                           'BUI_DC','initial_spread_index_transformed'], axis = 1)
y = new_data_encoded['fire_weather_index_transformed']
# Example usage
vif_df = calculate_vif(X)
print(vif_df)

  

                                Feature        VIF
0                           temperature  80.920233
1                 dew_point_temperature  79.527227
2                     relative_humidity  55.689530
3                  atmospheric_pressure   1.160642
4                         precipitation   1.911497
5                             rain_days   7.307396
6   fine_fuel_moisture_code_transformed  10.491272
7        duff_moisture_code_transformed   4.155615
8              drought_code_transformed   2.334213
9                wind_speed_transformed   1.671723
10                           FFMC__TEMP   4.559442
11                              FFMC_RH  18.984182
12                              RH_TEMP   3.450511
13                              RD_FFMC   9.283745
14                                DC_RD   3.235218
15                              WS_TENP  11.092800
16                               IST_WS  19.880624
17                             ISI_FFMC  64.126494
18                             

In [40]:
# Prepare features and target from your new test data
X = new_data_encoded.drop(['fire_weather_index_transformed','BUI_DMC','build_up_index_transformed','country_United States',
                           'BUI_DC','initial_spread_index_transformed','dew_point_temperature'], axis = 1)
y = new_data_encoded['fire_weather_index_transformed']
# Example usage
vif_df = calculate_vif(X)
print(vif_df)

                                Feature        VIF
0                           temperature   1.872911
1                     relative_humidity   3.098327
2                  atmospheric_pressure   1.160601
3                         precipitation   1.908597
4                             rain_days   7.305958
5   fine_fuel_moisture_code_transformed  10.484129
6        duff_moisture_code_transformed   4.155398
7              drought_code_transformed   2.334181
8                wind_speed_transformed   1.667479
9                            FFMC__TEMP   4.495123
10                              FFMC_RH  18.677299
11                              RH_TEMP   2.974616
12                              RD_FFMC   9.281563
13                                DC_RD   3.233684
14                              WS_TENP  11.063146
15                               IST_WS  19.749842
16                             ISI_FFMC  63.332156
17                               ISI_RH  19.071988
18                             